In [1]:
import numpy as np
import pandas as pd

Data loading

Data link : https://www.kaggle.com/datasets/arshid/iris-flower-dataset

In [2]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv("iris.csv", skiprows=1, header=None, names=col_names)
data.head(10)

,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
10,4.9,3.1,1.5,0.1,Iris-setosa


In [3]:
data.type.value_counts()   # count the number of  type

type
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

Node Class

In [4]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold  # for check condition 
        self.left = left   # to navigate
        self.right = right  # to navigate
        self.info_gain = info_gain  # for information gain from the dicision tree
        
        # for leaf node
        self.value = value

Tree Class

In [5]:
class DecisionTreeClassifier():
    def __init__(self, max_depth=2, min_samples_split=2):
        # Constructor
        
        #stoping condition 
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split 
        
        self.root = None  # initalize the root of the tree
        
        
    def build_tree(self,dataset,curr_depth=0):
        '''recursive function to build the tree '''
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        # Split until the stopping condition is met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            
            best_split = self.get_best_split(dataset, num_samples, num_features) # get the best split

            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)  # build the left subtree
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1) # build the right subtree
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
            

        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        
        return Node(value=leaf_value)
    
    def get_best_split(self,dataset, num_samples, num_features):
        '''function to get the best split'''
        best_split = {}
        max_info_gain = -float("inf")

        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values) # get unique values of the feature

            # loop over all the feature values
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)

                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y,left_y,right_y = dataset[:,-1],dataset_left[:,-1],dataset_right[:,-1]

                    # compute information gain
                    current_info_gain = self.information_gain(y, left_y, right_y, "gini")

                    # update the best split if needed
                    if current_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"]=dataset_right
                        best_split["info_gain"]=current_info_gain

                        max_info_gain = current_info_gain

        return best_split


    def split(self,dataset, feature_index, threshold):
        '''function to split the dataset'''
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])  # left dataset
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])  # right dataset
        return dataset_left, dataset_right
    

    def information_gain(self, parent, left_child, right_child, mode="entropy"):
        '''function to compute information gain'''
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)

        if mode=="gini":
            gain = self.gini_index(parent) - (weight_left*self.gini_index(left_child) + weight_right*self.gini_index(right_child))
        else:
            gain = self.entropy(parent) - (weight_left*self.entropy(left_child) + weight_right*self.entropy(right_child))
    
        return gain
    

    def gini_index(self, y):
        '''function to compute gini index'''
        classes = np.unique(y)
        gini = 0
        for cls in classes:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini

    def entropy(self, y):
        '''function to compute entropy'''
        entropy = 0 
        classes = np.unique(y)
        for cls in classes:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy


    def calculate_leaf_value(self, Y):
        '''function to compute leaf node'''
        Y = list(Y)
        return max(Y, key=Y.count)
    

    def print_tree(self, tree=None, indent=" "):
        '''function to print the tree'''
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", str(tree.threshold))
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)    


    def fit(self, X, Y):
        '''function to fit the tree'''  
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)


    def predict(self, X):
        '''function to predict the output'''

        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions


    def make_prediction(self, x, tree):
        '''function to make prediction'''
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x,tree.right)



Train - Test Split

In [6]:
from sklearn.model_selection import train_test_split

X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values.reshape(-1,1)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 41)


Fit the Model

In [7]:
classifier = DecisionTreeClassifier(max_depth=10)
classifier.fit(X_train, Y_train)
classifier.print_tree()

X_2 <= 1.9
 left:Iris-setosa
 right:X_3 <= 1.5
  left:X_2 <= 4.9
    left:Iris-versicolor
    right:Iris-virginica
  right:X_2 <= 5.0
    left:X_1 <= 2.8
        left:Iris-virginica
        right:X_0 <= 5.9
                left:Iris-versicolor
                right:X_0 <= 6.0
                                left:Iris-virginica
                                right:Iris-versicolor
    right:Iris-virginica


Test the Model

In [9]:
Y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy =accuracy_score(Y_test, Y_pred)

print(f'accuracy is {accuracy *100}  %')

accuracy is 90.0  %
